# Lecture 20 Supplemental Notebook

by Suraj Rampure, Josh Hug, John DeNero

**Note:** In this lecture, the majority of the narrative and content is in the slides. The notebook is almost entirely supplementary.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = (5, 3)
plt.rcParams['figure.dpi'] = 100
#plt.rcParams['lines.linewidth'] = 3
sns.set()

from scipy.optimize import minimize

import requests
import os

def fetch():
    path = 'nba.csv'
    if not os.path.exists(path):
        url = 'https://stats.nba.com/stats/leaguegamelog/'
        params = (
            ('Counter', '0'),
            ('DateFrom', ''),
            ('DateTo', ''),
            ('Direction', 'ASC'),
            ('LeagueID', '00'),
            ('PlayerOrTeam', 'T'),
            ('Season', '2017-18'),
            ('SeasonType', 'Regular Season'),
            ('Sorter', 'DATE'),
        )
        headers = {
            'User-Agent': 'PostmanRuntime/7.4.0'
        }
        response = requests.get(url, params=params, headers=headers)
        data = response.json()['resultSets'][0]
        df = pd.DataFrame(data=data['rowSet'], columns=data['headers'])
        df.to_csv(path, index=False)
        return df
    else:
        return pd.read_csv(path)
    
df = fetch()

## Minimizing Empirical Risk using Cross Entropy Loss

### Switching back to NBA data

Let's start off with the cleaned dataframe from the end of last lecture. 

In [ ]:
df.head()

In [ ]:
one_team = df.groupby("GAME_ID").first()
opponent = df.groupby("GAME_ID").last()
games = one_team.merge(opponent, left_index = True, right_index = True, suffixes = ["", "_OPP"])
games["FG_PCT_DIFF"] = games["FG_PCT"] - games["FG_PCT_OPP"]
games['WON'] = games['WL'].replace('L', 0).replace('W', 1)
games = games[['TEAM_NAME', 'MATCHUP', 'WON', 'FG_PCT_DIFF']]

In [ ]:
games.head()

Before going any further, we need a $\sigma(\cdot)$ defined.

In [ ]:
def sigma(t):
    return 1 / (1 + np.e**(-t))

First, let's try to predict `WON` from just `FG_PCT_DIFF`, as in last week's lecture. The below code determines the optimal $\hat{\beta}$ in this case, using MSE.

In [ ]:
def mse_loss_single_arg_nba(beta):
    x = games['FG_PCT_DIFF']
    y_obs = games['WON']
    y_hat = sigma(x * beta)
    return np.mean((y_obs - y_hat)**2)

In [ ]:
best_beta_mse_nba = minimize(mse_loss_single_arg_nba, x0 = 0)["x"][0]
best_beta_mse_nba

Note, that this beta is the minimizing value of 
$$R(\beta) = \frac{1}{n} \sum_{i = 1}^n (y_i - \sigma(x_i\beta))^2$$ where $x_i$ corresponds to `FG_PCT_DIFF` and $y_i$ corresponds to `WIN`.

Now that we have $\hat{\beta}_{MSE}$, let's compute $\hat{\beta}_{CE}$. To do that, we'll create a `cross_entropy_loss` and `cross_entropy_loss_single_arg_nba`. We will pass the latter into `scipy.optimize.minimize`.

In [ ]:
# Defining cross entropy loss (copied from above, repeated for clarity)
def cross_entropy_loss(y, y_hat):
    return -y * np.log(y_hat) - (1-y) * np.log(1-y_hat)

def cross_entropy_loss_single_arg_nba(beta):
    x = games['FG_PCT_DIFF']
    y_obs = games['WON']
    y_hat = sigma(beta * x)
    return np.mean(cross_entropy_loss(y_obs, y_hat))

In [ ]:
best_beta_ce_nba = minimize(cross_entropy_loss_single_arg_nba, x0 = 0)["x"][0]
print('optimal beta using mse on nba data: ', best_beta_mse_nba)
print('optimal beta using mean cross entropy on nba data: ', best_beta_ce_nba)

We see that $\hat{\beta}_{CE}$ is slightly greater than $\hat{\beta}_{MSE}$. That is, so long as our initial guess for $\vec{\beta}$ is in the right region, we're likely to end up with a $\hat{\beta}_{MSE}$ that's close to $\hat{\beta}_{CE}$. However, cross entropy loss is more likely to converge to a reasonable answer from a much wider range of initial guesses. (Due to other numerical issues, it's still possible that cross entropy loss doesn't actually converge. But, don't worry about that for our purposes.)

Let's use $\hat{\beta}_{CE}$ to do some prediction. Let's overlay our fitted model over a scatterplot of `WON` vs `FG_PCT_DIFF`.

In [ ]:
fg_pcts = np.linspace(-0.3, 0.3, 1000)
y_pred = sigma(fg_pcts * best_beta_ce_nba)

plt.xlabel('FG_PCT_DIFF')
plt.ylabel(r'$P(Y = 1 | x)$')
plt.plot(fg_pcts, y_pred, color = 'r')
plt.scatter(games['FG_PCT_DIFF'], games['WON'] + np.random.normal(0, 0.01, len(games['WON'])));

## Classification

Let's switch to a completely different dataset, and go through the entire process of fitting a model and using it to classify. We will be using `sklearn`'s built in `breast_cancer` dataset, a commonly used dataset for classification purposes. 

We will also split our data into train and test (as we always should). You will have an opportunity at the end of this notebook to test our model on the testing data, but we will not get to this point in lecture.

In [ ]:
import sklearn.datasets

In [ ]:
data_dict = sklearn.datasets.load_breast_cancer()
cancer = pd.DataFrame(data_dict['data'], columns=data_dict['feature_names'])
cancer['bias'] = 1.0
# Target data_dict['target'] = 0 is malignant; 1 is benign
cancer['malignant'] = 1 - data_dict['target']
cancer.iloc[0]

In [ ]:
cancer.describe()

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(cancer, test_size=0.25, random_state=100)
print("Training Data Size: ", len(train))
print("Test Data Size: ", len(test))

First, we scatter `malignant` vs. `mean radius`. 

In [ ]:
plt.xlabel('mean radius')
plt.ylabel('malignant')
plt.scatter(train['mean radius'], train['malignant']);

Now, as we did in the last lecture, let's take a look at the average `malignant` value for various bins of `mean radius`. That's what we'll plot in gold:

In [ ]:
radii = np.linspace(5, 30, 50)
averages = [np.average(train[np.abs(train['mean radius']-r)<2]['malignant']) for r in radii]
plt.xlabel('mean radius')
plt.ylabel('malignant')
plt.scatter(train['mean radius'], train['malignant']);
plt.scatter(radii, averages, color='gold');

In [ ]:
def features(t):
    return t[['bias', 'mean radius']].values.T
    
x_train, y_train = features(train), train['malignant'].values

In [ ]:
from sklearn.linear_model import LogisticRegression

We'll now use `sklearn.linear_model.LogisticRegression` to find the optimal $\vec{\hat{\beta}}$ here.

In [ ]:
model = LogisticRegression(fit_intercept=False, C=1e9, solver='lbfgs')
model.fit(x_train.T, y_train)
beta_2_features = model.coef_[0]
beta_2_features

Using this value of $\vec{\hat{\beta}}$, let's plot of $P(Y = 1 | \vec{x})$ in red, overlaid on our previous scatter plot:

In [ ]:
plt.scatter(train['mean radius'], train['malignant'], label = 'original data');
plt.xlabel('mean radius')
plt.ylabel('malignant')
plt.scatter(radii, averages, color='gold', label = 'binned means');
plt.plot(radii, sigma(beta_2_features[0] + radii * beta_2_features[1]), color='r', label = 'logistic model');
plt.legend();

Great, now we've fit a model using two features (`mean radius` plus an intercept term). Let's now...
1. Create a classifier (i.e. threshold our probabilities)
2. Compute the accuracy, precision, and recall of our model

In [ ]:
def predict_prob(X, y):
    model = LogisticRegression(fit_intercept = False, C=1e9, solver='lbfgs')
    model.fit(X.T, y)
    beta = model.coef_[0]
    
    return sigma(X.T @ beta)

# Default threshold is 0.5, but we can change it
def classify(vals, threshold = 0.5):
    return np.int64(vals >= threshold)

In [ ]:
classify(np.array(([0.2, 0.6, 0.4])))

In [ ]:
def accuracy(actual, pred):
    return np.mean(actual == pred)

def precision(actual, pred):
    # It's not necessary to define each of these in both the function for precision
    # and recall, but they're here just for the sake of clarity
    tp = sum((actual == pred) & (actual == 1))
    tn = sum((actual == pred) & (actual == 0))
    fp = sum((actual != pred) & (actual == 0))
    fn = sum((actual != pred) & (actual == 1))
    
    return tp / (tp + fp)

def recall(actual, pred):
    tp = sum((actual == pred) & (actual == 1))
    tn = sum((actual == pred) & (actual == 0))
    fp = sum((actual != pred) & (actual == 0))
    fn = sum((actual != pred) & (actual == 1))
    
    return tp / (tp + fn)

In [ ]:
probs = predict_prob(x_train, y_train)
y_pred = classify(probs)

In [ ]:
accuracy(y_train, y_pred)

In [ ]:
precision(y_train, y_pred)

In [ ]:
recall(y_train, y_pred)

Let's see how these values change as we adjust our threshold.

In [ ]:
accuracies = []
precisions = []
recalls = []
for t in range(1, 11, 1):
    threshold = t / 10
    y_pred = classify(probs, threshold)
    acc = accuracy(y_train, y_pred)
    pre = precision(y_train, y_pred)
    rec = recall(y_train, y_pred)
    accuracies.append(acc)
    precisions.append(pre)
    recalls.append(rec)
    print("threshold p >= {}: accuracy {}, precision {}, recall {}".format(threshold, np.round(acc, 2),
                                                                           np.round(pre, 2),
                                                                           np.round(rec, 2)))

In [ ]:
plt.xlabel('precision')
plt.ylabel('recall')
plt.xlim(0.5, 1)
plt.ylim(0.5, 1)
plt.title('Precision vs. Recall')
plt.scatter(precisions, recalls);

As we can see, as precision increases, recall decreases. This further emphasizes the point from lecture that there's a "tradeoff" between precision and recall.

In [ ]:
plt.xlabel('threshold * 10')
plt.ylabel('accuracy')
plt.plot(list(range(1, 11, 1)), accuracies);

On the otherhand, accuracy seems to peak near $t = 0.6$ as our threshold.

**What we really should be doing is looking at testing accuracy, though, since we split our data into `train` and `test`.**
We won't have time to get to this during lecture, but you should try and compute the testing accuracy, precision, and recall of our `malignant` classifier, and see how that changes with our threshold. (You will get some practice with these ideas in Project 2.)